# Text2Everything SDK - Complete End-to-End Example

This notebook demonstrates a complete workflow using the Text2Everything SDK:
**Project → Context → Golden Example → Schema → Connector → Session → Chat → Feedback**

## Prerequisites

1. Install the SDK: `pip install text2everything-sdk`
2. Have your API endpoint and API key ready
3. Ensure you have access to the Text2Everything API
4. Have a database connection available for the connector example

## Setup and Configuration

In [ ]:
import sys
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Install the Text2Everything SDK (local installation)
!{sys.executable} -m pip install --upgrade ./text2everything_sdk-0.1.6-py3-none-any.whl

In [ ]:
# Import the SDK
from text2everything_sdk import Text2EverythingClient
from text2everything_sdk.exceptions import (
    AuthenticationError,
    ValidationError,
    NotFoundError,
    RateLimitError
)

print("✅ SDK imported successfully!")

# Configuration
ACCESS_TOKEN = ""  # Replace with your OIDC access token
WORKSPACE_NAME = ""  # Optional workspace name

## Initialize the Client

In [ ]:
# Initialize the Text2Everything client
client = Text2EverythingClient(
    access_token=ACCESS_TOKEN,
    workspace_name=WORKSPACE_NAME,
)

print("✅ Client initialized successfully!")

## 1. 🆕 Projects - Simplified Creation

In [ ]:
# 🆕 NEW: Direct keyword arguments
project = client.projects.create(
    name="my-first-project",
    description="My first project"
)

print(f"Project ID: {project.id}")
print(f"Project Name: {project.name}")

PROJECT_ID = project.id

## 2. 🆕 Contexts - Simplified Creation

In [ ]:
context = client.contexts.create(
    project_id=PROJECT_ID,
    name="E-commerce Business Rules",
    description="Business rules and definitions for e-commerce analytics",
    content="""Business Rules:
- Revenue = quantity * unit_price
- Active customers have made a purchase in the last 90 days
- High-value customers have lifetime value > $1000

Definitions:
- LTV: Customer Lifetime Value
- AOV: Average Order Value
- Churn: Customers who haven't purchased in 180+ days
""",
    is_always_displayed=False
)

print(f"Context ID: {context.id}")
print(f"Always displayed: {context.is_always_displayed}")

## 3. 🆕 Golden Examples - Simplified Creation & Bulk Operations

In [ ]:
example1 = client.golden_examples.create(
    project_id=PROJECT_ID,
    user_query="What is our total revenue?",
    sql_query="SELECT SUM(quantity * unit_price) as total_revenue FROM orders WHERE status = 'completed';",
    description="Calculate total revenue from all completed orders",
    is_always_displayed=False
)

print("✅ Golden example created with new API pattern!")
print(f"Example ID: {example1.id}")

# 🆕 NEW: Bulk creation with list of dictionaries
examples_data = [
    {
        "user_query": "Who are our top 10 customers by total spending?",
        "sql_query": "SELECT c.customer_id, c.name, SUM(o.quantity * o.unit_price) as total_spent FROM customers c JOIN orders o ON c.customer_id = o.customer_id WHERE o.status = 'completed' GROUP BY c.customer_id, c.name ORDER BY total_spent DESC LIMIT 10;",
        "description": "Find top 10 customers by total spending"
    },
    {
        "user_query": "Show me monthly sales for this year",
        "sql_query": "SELECT DATE_TRUNC('month', order_date) as month, SUM(quantity * unit_price) as monthly_revenue FROM orders WHERE status = 'completed' AND EXTRACT(YEAR FROM order_date) = EXTRACT(YEAR FROM CURRENT_DATE) GROUP BY DATE_TRUNC('month', order_date) ORDER BY month;",
        "description": "Monthly sales trend for the current year"
    }
]

bulk_examples = client.golden_examples.bulk_create(
    project_id=PROJECT_ID,
    golden_examples=examples_data
)

print(f"✅ Bulk created {len(bulk_examples)} examples with new API pattern!")

## 4. 🆕 Schema Metadata - Simplified Creation & Bulk Operations

In [ ]:
# 🆕 NEW: Individual creation with keyword arguments
customers_schema = client.schema_metadata.create(
    project_id=PROJECT_ID,
    name="customers_table",
    description="Customer information table",
    schema_data={
        "table": {
            "name": "customers",
            "columns": [
                {"name": "customer_id", "type": "INTEGER", "primary_key": True},
                {"name": "name", "type": "VARCHAR(100)", "nullable": False},
                {"name": "email", "type": "VARCHAR(255)", "unique": True},
                {"name": "registration_date", "type": "DATE"},
                {"name": "is_active", "type": "BOOLEAN", "default": True}
            ]
        }
    },
    is_always_displayed=False,
    validate=True # we like to validate this complex structure
)

print(f"Schema ID: {customers_schema.id}")

# 🆕 NEW: Bulk creation with list of dictionaries
schemas_data = [
    {
        "name": "orders_table",
        "description": "Order transactions table",
        "schema_data": {
            "table": {
                "name": "orders",
                "columns": [
                    {"name": "order_id", "type": "INTEGER", "primary_key": True},
                    {"name": "customer_id", "type": "INTEGER", "foreign_key": "customers.customer_id"},
                    {"name": "product_id", "type": "INTEGER", "foreign_key": "products.product_id"},
                    {"name": "quantity", "type": "INTEGER"},
                    {"name": "unit_price", "type": "DECIMAL(10,2)"},
                    {"name": "order_date", "type": "TIMESTAMP"},
                    {"name": "status", "type": "VARCHAR(20)", "default": "pending"}
                ]
            }
        }
    },
    {
        "name": "products_table",
        "description": "Product catalog table",
        "schema_data": {
            "table": {
                "name": "products",
                "columns": [
                    {"name": "product_id", "type": "INTEGER", "primary_key": True},
                    {"name": "name", "type": "VARCHAR(200)"},
                    {"name": "category", "type": "VARCHAR(50)"},
                    {"name": "price", "type": "DECIMAL(10,2)"},
                    {"name": "in_stock", "type": "BOOLEAN"}
                ]
            }
        }
    }
]

bulk_schemas = client.schema_metadata.bulk_create(
    project_id=PROJECT_ID,
    schema_metadata_list=schemas_data,
    validate=True  # we like to validate this complex structure
)

print(f"✅ Bulk created {len(bulk_schemas)} schemas!")

## 5. 🆕 Connectors - Simplified Creation with Smart Defaults

In [ ]:
# Using Snowflake connector
connector = client.connectors.create(
    name="snowflake-connector",
    description="Snowflake connector for project My First Project",
    db_type="snowflake",  # Smart default: port=443
    host=os.getenv("SNOWFLAKE_HOST"),
    username=os.getenv("SNOWFLAKE_USERNAME"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    config={
        "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
        "role": os.getenv("SNOWFLAKE_ROLE")
    }
)

print("✅ Connector created with new API pattern!")
print(f"Connector ID: {connector.id}")
print(f"Database Type: {connector.db_type}")
print(f"Host: {connector.host}:{connector.port}")
print(f"🆕 Smart default port: {connector.port} (auto-detected for {connector.db_type})")
print(f"Database: {connector.database}")
if hasattr(connector, 'config') and connector.config:
    print(f"Warehouse: {connector.config.get('warehouse', 'N/A')}")
    print(f"Role: {connector.config.get('role', 'N/A')}")

CONNECTOR_ID = connector.id

## 6. 🆕 Chat Sessions - Simplified Creation

In [ ]:
session = client.chat_sessions.create(
    project_id=PROJECT_ID,
    name="Testing 1212",
    description="Testing 12 12"
)

print(f"Session ID: {session.id}")
print(f"Session Name: {session.name}")

SESSION_ID = session.id

## 7. 🆕 Chat Methods - Simplified Text-to-SQL

In [ ]:
sql_response = client.chat.chat_to_sql(
    project_id=PROJECT_ID,
    chat_session_id=SESSION_ID,
    query="Cuantas suscripciones el mes pasado?"
    # contexts_limit=10,
    # examples_limit=10,
    # schema_limit=60
)

print(f"Generated SQL: {sql_response.sql_query}")

## 1.7. 🆕 Executions -  Chat-to-Answer

In [ ]:
answer_response = client.chat.chat_to_answer(
    project_id=PROJECT_ID,
    chat_session_id=SESSION_ID,
    query="What's the database version",
    connector_id=CONNECTOR_ID,  # the resulting sql query gets executed against the connector
    contexts_limit=1,
    examples_limit=1,
    schema_limit=1
)

print(f"Answer: {answer_response.execution_result.result if hasattr(answer_response, 'execution_result') else 'N/A'}")

## 8. 🆕 Executions - Simplified SQL Execution

In [ ]:
# 🆕 NEW: execute_sql with keyword arguments
execution_result = client.executions.execute_sql(
    connector_id=CONNECTOR_ID,
    chat_message_id=sql_response.id,
    sql_query="SELECT COUNT(*) as customer_count FROM customers WHERE is_active = true;",
    chat_session_id="session_123"
)

print("✅ SQL executed with new API pattern!")
print(f"Execution ID: {execution_result.id}")
print(f"Status: {execution_result.status}")
if execution_result.results:
    print(f"Results: {execution_result.results}")

## 9. 🆕 Feedback - Simplified Feedback Creation

In [ ]:
positive_feedback = client.feedback.create(
    project_id=PROJECT_ID,
    chat_message_id=sql_response.id,
    feedback="Perfect SQL query! Exactly what I needed.",
    is_positive=True,
    execution_id=execution_result.execution_id
)

print(f"Feedback ID: {positive_feedback.id}")
print(f"Is Positive: {positive_feedback.is_positive}")


negative_feedback = client.feedback.create_negative(
    project_id=PROJECT_ID,
    chat_message_id=sql_response.id,
    feedback="Query could be improved with better filtering."
)

print(f"Feedback ID: {negative_feedback.id}")

## 10. 🆕 Update Operations - Simplified Updates

In [ ]:
# Update project
updated_project = client.projects.update(
    project_id=PROJECT_ID,
    name="E-commerce Analytics - Updated",
    description="Updated description with new insights"
)


print(f"New name: {updated_project.name}")

# Update connector
updated_connector = client.connectors.update(
    connector_id=CONNECTOR_ID,
    name="E-commerce Database - Production",
    description="Updated production database connector"
)

print("✅ Connector updated with new API pattern!")
print(f"New name: {updated_connector.name}")

# Update schema metadata
updated_schema = client.schema_metadata.update(
    schema_id=customers_schema.id,
    name="customers_table_v2",
    description="Updated customer information table with new fields"
)

print(f"New name: {updated_schema.name}")

In [ ]:
# Clean up
client.close()
print("✅ Client connection closed")
print("\n🎉 API Patterns Demo Complete!")